In [1]:
import os
import sys
import numpy as np
import matplotlib.pyplot as plt
from scipy.io import loadmat, savemat
import tifffile
import cv2
import random
import datetime
import time

from PyQt5.QtWidgets import *
from PyQt5.QtGui import QPixmap, QImage, QPainter, QPen, QColor, QFont, QPainterPath, QBrush
from PyQt5.QtCore import Qt, QTimer, QItemSelection
from matplotlib.backends.backend_qt5agg import FigureCanvasQTAgg as FigureCanvas
from matplotlib.figure import Figure

dir_notebook = os.path.dirname(os.path.abspath("__file__"))
# 親ディレクトリのパスを取得
dir_parent = os.path.dirname(dir_notebook)
if not dir_parent in sys.path:
    sys.path.append(dir_parent)

from optic.config import *
from optic.controls import *
from optic.manager import *
from optic.gui import *
from optic.io import *
from optic.utils import *
from optic.preprocessing import *
from optic.visualization import *

class Suite2pROITrackingGUI(QMainWindow):
    def __init__(self):
        APP_NAME = "SUITE2P_ROI_TRACKING"
        QMainWindow.__init__(self)
        self.widget_manager, self.config_manager, self.data_manager, self.control_manager = initManagers(WidgetManager(), ConfigManager(), DataManager(), ControlManager())
        self.config_manager.setCurrentApp(APP_NAME)
        self.app_keys = self.config_manager.gui_defaults["APP_KEYS"]
        self.app_key_pri = self.app_keys[0]

        self.setupUI_done = False
        setupMainWindow(self, self.config_manager.gui_defaults)

        self.initUI()

    """
    setup UI Function
    """
    def initUI(self):
        self.central_widget = QWidget(self)
        self.setCentralWidget(self.central_widget)
        self.layout_main = QGridLayout(self.central_widget)

        # FileLoadUI用のレイアウト
        self.layout_file_load = QVBoxLayout()
        self.setupFileLoadUI()
        self.layout_main.addLayout(self.layout_file_load, 1, 0, 1, 1)

        # メインUI用のレイアウト
        self.layout_main_ui = QGridLayout()
        self.layout_main.addLayout(self.layout_main_ui, 0, 0, 1, 1)

    def setupFileLoadUI(self):
        file_load_widget = QWidget()
        layout = QVBoxLayout(file_load_widget)
        # ファイル読み込み用のUIを追加
        layout.addLayout(self.makeLayoutSectionBottom())
        # bindFunc
        self.bindFuncFileLoadUI()

        self.layout_file_load.addWidget(file_load_widget)

    def loadFilePathsandInitialize(self):
        self.control_manager, self.data_manager = initManagers(self.control_manager, self.data_manager)
        success = self.loadData()
        if success:
            self.setupMainUI()
        else:
            return

    def setupMainUI(self):
        if self.setupUI_done:
            # メインUIのクリア
            clearLayout(self.layout_main_ui)
        
        # 新しいメインUIの設定
        self.setupMainUILayouts()
        self.setupControls()
        self.bindFuncAllWidget()

        self.setupUI_done = True

    def loadData(self):
        pass
        # success = loadFallMATWithGUI(
        #     q_window=self, 
        #     data_manager=self.data_manager, 
        #     key_app=self.app_key_pri, 
        #     path_fall=self.widget_manager.dict_lineedit[f"{self.app_key_pri}_path_fall"].text()
        # )
        # if self.widget_manager.dict_lineedit[f"{self.app_key_pri}_path_reftif"].text() != "":
        #     self.data_manager.dict_im_bg_chan2[self.app_key_pri] = loadTIFImage(
        #         data_manager=self.data_manager, 
        #         key_dict_im_chan2=self.app_key_pri, 
        #         path_image=self.widget_manager.dict_lineedit[f"{self.app_key_pri}_path_reftif"].text(), 
        #         preprocessing=True
        #         )
        # return success

    def setupMainUILayouts(self):
        self.layout_main_ui.addLayout(self.makeLayoutSectionLeftUpper(), 0, 0, 1, 1)
        self.layout_main_ui.addLayout(self.makeLayoutSectionRightUpper(), 0, 1, 1, 1)
        self.layout_main_ui.addLayout(self.makeLayoutSectionCentral(), 1, 0, 1, 2)

    def setupControls(self):
        pass
        # self.control_manager.table_controls[self.app_key_pri] = TableControl(
        #                                                         key_app=self.app_key_pri,
        #                                                         q_table=self.widget_manager.dict_table[self.app_key_pri],
        #                                                         data_manager=self.data_manager,
        #                                                         widget_manager=self.widget_manager,
        #                                                         config_manager=self.config_manager,
        #                                                         control_manager=self.control_manager,
        #                                                         )
        
        # self.control_manager.table_controls[self.app_key_pri].setupWidgetROITable(self.app_key_pri)
        # self.control_manager.view_controls[self.app_key_pri] = ViewControl(
        #                                                             key_app=self.app_key_pri,
        #                                                             q_view=self.widget_manager.dict_view[self.app_key_pri], 
        #                                                             q_scene=self.widget_manager.dict_scene[self.app_key_pri], 
        #                                                             data_manager=self.data_manager, 
        #                                                             widget_manager=self.widget_manager,
        #                                                             config_manager=self.config_manager,
        #                                                             control_manager=self.control_manager,
        #                                                             )
        # self.control_manager.view_controls[self.app_key_pri].setViewSize()

    """
    makeLayout Function; Component
    小要素のLayout
    return -> Layout
    """
    "Left Upper"

    "Right Upper"

    "Central"

    "Bottom"
    # ファイル読み込み用UI Layout
    def makeLayoutComponentFileLoadUI(self):
        layout = QVBoxLayout()

        # LineEdit
        for key_app in self.app_keys:
            list_label = [f"Fall mat file path ({key_app} Image)", "Reference Tiff image file path ({key_app} Image) (optional)"]
            list_key = [f"{key_app}_path_fall", f"{key_app}_path_reftif"]
            for label, key in zip(list_label, list_key):
                layout.addLayout(makeLayoutLoadFileWidget(self.widget_manager, label=label, key_label=key, key_lineedit=key, key_button=key))
        # Button
        layout.addLayout(makeLayoutLoadFileExitHelp(self.widget_manager))
        return layout

    

    """
    makeLayout Function; Section
    領域レベルの大Layout
    """
    # 上段, 左
    def makeLayoutSectionLeftUpper(self):
        layout = QVBoxLayout()
        return layout
    
    # 上段. 右
    def makeLayoutSectionRightUpper(self):
        layout = QVBoxLayout()
        return layout
    
    # 中段
    def makeLayoutSectionCentral(self):
        layout = QVBoxLayout()
        return layout

    # 下段
    def makeLayoutSectionBottom(self):
        layout = self.makeLayoutComponentFileLoadUI()
        return layout

    """
    bindFunc Function
    配置したwidgetに関数を紐づけ
    """
    def bindFuncFileLoadUI(self):
        for key_app in self.app_keys:
            list_key = [f"{key_app}_path_fall", f"{key_app}_path_reftif"]
            list_filetype = ["mat", "tiff"]
            for key, filetype in zip(list_key, list_filetype):
                bindFuncLoadFileWidget(q_widget=self, q_button=self.widget_manager.dict_button[key], q_lineedit=self.widget_manager.dict_lineedit[key], filetype=filetype)

        self.widget_manager.dict_button["load_file"].clicked.connect(lambda: self.loadFilePathsandInitialize())
        bindFuncExit(q_window=self, q_button=self.widget_manager.dict_button["exit"])

    def bindFuncAllWidget(self):
        pass


if __name__ == "__main__":
    app = QApplication(sys.argv) if QApplication.instance() is None else QApplication.instance()
    gui = Suite2pROITrackingGUI()
    gui.show()
    sys.exit(app.exec_())

SystemExit: 0

c:\Users\HyperPC_Smee\anaconda3\envs\itkelastix\lib\site-packages\IPython\core\interactiveshell.py:3516: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [5]:
getattr(GuiDefaults, "SUITE2P_ROI_TRACKING")

{'MIN_PLOT_RANGE': 30,
 'WINDOW_SETTINGS': {'INIT_POSITION_X': 100,
  'INIT_POSITION_Y': 100,
  'WIDTH': 1200,
  'HEIGHT': 200,
  'MAX_WIDTH': 1920,
  'MAX_HEIGHT': 1080},
 'TITLE': 'Suite2pROITrackingGUI',
 'APP_KEYS': ['pri', 'sec'],
 'CHANNELS': ['Green', 'Red', 'Blue'],
 'VIEW_SETTINGS': {'DEFAULT_CONTRAST_MIN': 0, 'DEFAULT_CONTRAST_MAX': 255},
 'ROI_THRESHOLDS': {'npix': '(50, 200)',
  'radius': '(3, 12)',
  'aspect_ratio': '(0, 1.5)',
  'compact': '(0, 1.5)',
  'skew': '(1, 100)',
  'std': '(0, 100)'},
 'ROI_VISUAL_SETTINGS': {'COLOR_MIN': 100,
  'COLOR_MAX': 255,
  'DEFAULT_ROI_OPACITY': 128,
  'DEFAULT_HIGHLIGHT_OPACITY': 255},
 'TABLE_COLUMNS': {'pri': {'Cell ID': {'order': 0,
    'type': 'id',
    'width': 100,
    'editable': False},
   'Astrocyte': {'order': 1,
    'type': 'radio',
    'width': 100,
    'group': 'celltype',
    'default': False},
   'Neuron': {'order': 2,
    'type': 'radio',
    'width': 100,
    'group': 'celltype',
    'default': True},
   'Not Cell': {'